In [1]:
import requests
import json
import pandas as pd
import time
import convex as cx
import tmqa3 as tmqa

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [2]:
conversations_path = "/data/users/romain.claret/tm/mse.tm.chatbot.base/data/convex/test_set/test_set_ALL.json"
with open(conversations_path, "r") as data:
    conversations = json.load(data)

In [3]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/convex/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Saving Dataframe Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [4]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [5]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    try:
                        ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                    except:
                        return False
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [6]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
#ask_convex("100% senorita is a television show in what language?")

In [7]:
def ask_tmqa(question, verbose=False, timer=False, show_graph=False, cores=tmqa.mp.cpu_count(), banning_str=False):
    tmqa_result = tmqa.answer_question(question, verbose=verbose, timer=timer, show_graph=show_graph, cores=cores, banning_str=banning_str)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    #if tmqa_result[]
    #if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
    #    return False
    return tmqa_result[0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_tmqa("Who is the wife of Barack Obama")
#print(answer)

In [8]:
#HEADERS = ['question', 'answer', 'qanswer', 'platypus', 'convex',
#           'tm', "tm_time", "tm_top2", "tm_top3", "tm_top4", "tm_top5", "tm_topall"]
#df = pd.DataFrame(columns=HEADERS)
#LOADING
df = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/convex/benchmarking-qanswer-platypus-convex-qagraph-from-0-to-332.pickle.bz2")


In [9]:
for conversation in conversations:
    questions = [turn['question'] for turn in conversation['questions']]
    print(questions)
    golden_answers = [tmqa.wikidata_url_to_wikidata_id(turn['answer']) for turn in conversation['questions']]
    print(golden_answers)
    
    break

['What is the name of the writer of The Secret Garden?', 'Where does the story take place?', 'When was the book published?', 'What kind of book is it?', 'What magazine first published the book?']
['Q276028', 'Q163', '1910-01-01T00:00:00Z', 'Q1436734', 'Q3985643']


In [ ]:
### Evaluate
banning_str = False

start_time = time.time()
conversations_len = len(conversations)

for i, conversation in enumerate(conversations):
    if i >= len(df):
        questions = [turn['question'] for turn in conversation['questions']]
        question = questions[0]
        answers = [tmqa.wikidata_url_to_wikidata_id(turn['answer']) for turn in conversation['questions']]
        answer = answers[0]
        print("\r\t>>> Processing {}/{}: ({}) {}".format(i,conversations_len,answer,question), end='                                  ')
        
        #print("Asking qAnswer")
        result_qanswer = False #ask_qanswer(question)
        print("Asking Platypus")
        result_platypus = ask_platypus(question)
        print("Asking Convex")
        result_convex = ask_convex(question)
        print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == answer else False
        print("\n",i, "-> Platypus", result_platypus) if result_platypus == answer else False
        print("\n",i, "-> Convex", result_convex) if result_convex == answer else False

        df_tm = False
        df_tm_top2 = False
        df_tm_top3 = False
        df_tm_top4 = False
        df_tm_top5 = False
        df_tm_topall = False
        
        print("Asking GraphQA")
        start_time_tmqa = time.time()
        result_tmqa = ask_tmqa(question, verbose=True, timer=True, banning_str=banning_str)   
        if result_tmqa:
            df_tm = result_tmqa[0]
            if answer in result_tmqa[:2]:
                df_tm_top2 = True
            if answer in result_tmqa[:3]:
                df_tm_top3 = True
            if answer in result_tmqa[:4]:
                df_tm_top4 = True
            if answer in result_tmqa[:5]:
                df_tm_top5 = True
            if answer in result_tmqa:
                df_tm_topall = True
            
        df_tm_time = round(time.time()-start_time_tmqa,2)
        
        df = df.append({"question":question, 'answer':answer, 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex,
                       "tm":df_tm, "tm_time":df_tm_time, "tm_top2":df_tm_top2, "tm_top3":df_tm_top3, "tm_top4":df_tm_top4, "tm_top5":df_tm_top5, "tm_topall":df_tm_topall},
                       ignore_index=True)

        print("\n",i, "-> tmqa", df_tm) if str(df_tm) == str(answer) else False
        print("\n",i, "-> tmqa in answers", result_tmqa) if df_tm_topall == True else False
        
        print(df.tail(1))
        
        pickle_data(df, "benchmarking-qanswer-platypus-convex-qagraph-from-0-to-"+str(len(df)))
        
        print("\n")
    
    #break

print("->\tRunning time is {}s".format(round(time.time()-start_time,2)))
df.tail(5)

	>>> Processing 332/2240: (Q169889) On which network did Lost originally air?                                  Asking Platypus
Asking Convex

 332 -> Convex Q169889
Asking GraphQA
User input: On which network did Lost originally air?
--> Auto correcting question in progress...

-> Auto corrected q_nlp: On which network did Lost originally air 
-> q_themes: ([(air, ['Q7391292', 'Q318452']), (Air, ['Q11189065', 'Q11190093']), (Network, ['Q7000691', 'Q572165']), (network, ['Q15993745', 'Q1900326'])], [which network, originally air, On which network did Lost, Originally Air])
-> q_themes_enhanced: [('originally', ['Q53737447']), ('Originally', ['Q7102610']), ('Lost', ['Q223446']), ('lost', ['P1356']), ('Lose', ['Q33120323'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: network
behold: get_most_similar started with: air
-> q_predicates: [(did, ['P248']), (Lost, ['P1356']), (network, []), (air, [])]
-> q_predica

Asking Convex

 334 -> Convex Q51511
Asking GraphQA
User input: Who directed Night of the Living Dead?
--> Auto correcting question in progress...
-> Auto corrected q_nlp: Who directed Night of the Living Dead 
-> q_themes: ([(Night of the Living Dead, ['Q623051']), (Night, ['Q575', 'Q592503']), (the Living Dead, ['Q7747842', 'Q27877445']), (Night of the Living, ['Q545417', 'Q2652860']), (Living Dead, ['Q3461339', 'Q7747841']), (the living, ['Q20504012', 'Q7747823'])], [directed Night])
-> q_themes_enhanced: [('direct', ['Q15304504']), ('night', ['Q575']), ('Direct', ['Q1187369'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
-> q_predicates: [(directed, ['P57'])]
-> q_predicates 	Running time is 9.01s
-> q_focused_parts: [(Night, ['Q575', 'Q592503']), (Dead, ['Q1180781', 'Q15874429', 'Q2410746', 'Q504434']), (the living, ['Q20504012', 'Q7747823'])]
-> Building the graph with k_deep 3 ... (could be long)
->New graph 	Running time is 30.58s
-->  

->New graph 	Running time is 20.3s
-->  89 nodes and 94 edges
--> Removing meaningless subgraphs
--> New graph of: 76 nodes and 82 edges
---> Rebuilding the graph with k_deep 10 ... Previously: 76 nodes or 82 edges was below the limit of 100
->New graph 	Running time is 20.78s
-->  89 nodes and 94 edges
--> Removing meaningless subgraphs
--> New graph of: 76 nodes and 82 edges
---> Loop detected, returning the graph in the current state
-> predicates_dict: {'P57': 1, 'P31': 2, 'P3831': 1, 'P3092': 1, 'P1040': 1, 'P136': 3, 'P156': 2, 'P344': 1, 'P2363': 1, 'P155': 2, 'P2453': 11, 'P805': 1, 'P1411': 5, 'P1346': 4, 'P166': 1, 'P279': 4, 'P291': 2, 'P577': 2, 'P1995': 2}
-> paths_keywords: (['interstellar'], {'director': [director, ['P57']]}, [Who])
-> Computing possible paths... (could be long)
--> len(path_nodes): 6
->Computing possible paths 	Running time is 19.65s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 4
->	Running time is 3.17s
-> Computing hypothesises

--> len(path_nodes): 242
->Computing possible paths 	Running time is 10.31s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 206
->	Running time is 2.98s
-> Computing hypothesises...
--> hypothesises: [['Q1284058', 7.798615232260388], ['Q7752539', 3.614328159225882], ['Q46047837', 1.1471439007813211], ['Q19860854', 1.0573948886623195], ['Q272404', 0.825740027230818], ['Q1762059', 0.804846999627669], ['Q41253', 0.7522725649841601], ['Q4830453', 0.6134759428127], ['Q1860', 0.5971230160874398], ['Q1107679', 0.5890795085219132]]
->Computing hypothesises 	Running time is 43.6s
-> Computing golden paths...
--> len(golden_paths): 28
->	Running time is 10.97s
--> len(cleared_golden_paths): 16
---> First path: ['Q1284058', 'P57', 'Q7752539', 'P364', 'Q1860', 'P407', 'Q127552', 'P31', 'Q1762059']
->	Total Running time is 218.77s

                            question   answer qanswer platypus convex  \
338  What was the first Pixar movie?  Q171048   False  Q387066  False   

 

->New graph 	Running time is 10.51s
-->  86 nodes and 82 edges
--> Removing meaningless subgraphs
--> New graph of: 86 nodes and 82 edges
---> Rebuilding the graph with k_deep 6 ... Previously: 86 nodes or 82 edges was below the limit of 100
->New graph 	Running time is 10.45s
-->  92 nodes and 88 edges
--> Removing meaningless subgraphs
--> New graph of: 92 nodes and 88 edges
---> Rebuilding the graph with k_deep 7 ... Previously: 92 nodes or 88 edges was below the limit of 100
->New graph 	Running time is 10.5s
-->  96 nodes and 92 edges
--> Removing meaningless subgraphs
--> New graph of: 96 nodes and 92 edges
---> Rebuilding the graph with k_deep 8 ... Previously: 96 nodes or 92 edges was below the limit of 100
->New graph 	Running time is 10.45s
-->  102 nodes and 98 edges
--> Removing meaningless subgraphs
--> New graph of: 102 nodes and 98 edges
---> Rebuilding the graph with k_deep 9 ... Previously: 102 nodes or 98 edges was below the limit of 100
->New graph 	Running time is 1

--> len(path_nodes): 18608
->Computing possible paths 	Running time is 26.45s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 16996
->	Running time is 39.43s
-> Computing hypothesises...


In [ ]:
len(df)

In [ ]:
#result_convex

In [ ]:
df.tail(10)

In [ ]:
#SAVING
#pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-tm1-from-0-to-"+str(len(df_loaded)))

In [ ]:
#LOADING
#df_loaded = pd.read_pickle("/data/users/romain.claret/tm/wikidata-simplequestions/benchmark_pickles/benchmarking-qanswer-platypus-convex-tm1-from-0-to-9961.pickle.bz2")

In [ ]:
#df = df_loaded

In [ ]:
#df_loaded

In [ ]:
#df_loaded = df_loaded.replace("", False)

In [ ]:
#df_loaded['qanswer'][34] = False

In [ ]:
#del df_loaded['tm2']

In [ ]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [ ]:
#df_loaded['tm1_top4'] = "False"

In [ ]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [ ]:
#df_loaded

In [ ]:
#print(df.dtypes)

In [ ]:
#df_loaded_len = len(df_loaded)
#for i, question in enumerate(df_loaded['question']):
#    if i >= 0:
#    #if i >= 497:
#        source = str(df_loaded['source'][i])
#        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
#        
#        start_time = time.time()
#        result_tmqa_1 = ask_tmqa_1(question, verbose=True)
#        
#        if result_tmqa_1:
#            df_loaded['tm1'][i] = result_tmqa_1[0]
#            if source in result_tmqa_1[:2]:
#                df_loaded['tm1_top2'][i] = True
#            if source in result_tmqa_1[:3]:
#                df_loaded['tm1_top3'][i] = True
#            if source in result_tmqa_1[:4]:
#                df_loaded['tm1_top4'][i] = True
#            if source in result_tmqa_1[:5]:
#                df_loaded['tm1_top5'][i] = True
#            if source in result_tmqa_1:
#                df_loaded['tm1_topall'][i] = True
#        else:
#            df_loaded['tm1'][i] = False
#        end_time = time.time()
#        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
#        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
#        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa_1:",str(result_tmqa_1)+"\n")

In [ ]:
#df.tail(30)

In [ ]:
#df_loaded.head(30)

In [ ]:
#df = df_loaded.copy()
#df = df.replace("", False)

In [ ]:
#df_backup = df

In [ ]:
#max_row = 496
#df_len = len(df)
#df_qanswer_max = df[(df.index<=max_row) & (df.qanswer == df.source)]
#df_qanswer_max_len = len(df_qanswer_max)
#
#df_platypus_max = df[(df.index<=max_row) & (df.platypus == df.source)]
#df_platypus_max_len = len(df_platypus_max)
#
#df_convex_max = df[(df.index<=max_row) & (df.convex == df.source)]
#df_convex_max_len = len(df_convex_max)
#
#df_tm1_max = df[(df.index<=max_row) & (df.tm1 == df.source)]
#df_tm1_max_len = len(df_tm1_max)
#
#df_tm1_max_top2 = df[(df.index<=max_row) & (df.tm1_top2 == True)]
#df_tm1_max_top2_len = len(df_tm1_max_top2)
#
#df_tm1_max_top3 = df[(df.index<=max_row) & (df.tm1_top3 == True)]
#df_tm1_max_top3_len = len(df_tm1_max_top3)
#
#df_tm1_max_top4 = df[(df.index<=max_row) & (df.tm1_top4 == True)]
#df_tm1_max_top4_len = len(df_tm1_max_top4)
#
#df_tm1_max_top5 = df[(df.index<=max_row) & (df.tm1_top5 == True)]
#df_tm1_max_top5_len = len(df_tm1_max_top5)
#
#df_tm1_max_topall = df[(df.index<=max_row) & (df.tm1_topall == True)]
#df_tm1_max_topall_len = len(df_tm1_max_topall)
#
#print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
#print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
#print("convex:", df_convex_max_len, df_convex_max_len/max_row)
#print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
#print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
#print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
#print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
#print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
#print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)
#
#df[ & (df.qanswer == df.source)]

In [ ]:
#print("qanswer:",len(df[df.qanswer == df.source]),len(df[df.qanswer == df.source])/len(df))
#print("platypus:",len(df[df.platypus == df.source]),len(df[df.platypus == df.source])/len(df))
#print("convex:",len(df[df.convex == df.source]),len(df[df.convex == df.source])/len(df))
#print("tm1:",len(df[df.tm1 == df.source]),len(df[df.tm1 == df.source])/len(df))
#print("tm1_top2:",len(df[df.tm1_top2 == True]),len(df[df.tm1_top2 == True])/len(df))
#print("tm1_top3:",len(df[df.tm1_top3 == True]),len(df[df.tm1_top3 == True])/len(df))
#print("tm1_top4:",len(df[df.tm1_top4 == True]),len(df[df.tm1_top4 == True])/len(df))
#print("tm1_top5:",len(df[df.tm1_top5 == True]),len(df[df.tm1_top5 == True])/len(df))
#print("tm1_topall:",len(df[df.tm1_topall == True]),len(df[df.tm1_topall == True])/len(df))

In [ ]:
#df.tm1_top2 = df.tm1_top3